## In this Step:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


In [1]:
import pandas as pd

### Store the dataset in database table and connect to database

In [2]:
import csv, sqlite3

con = sqlite3.connect("SpaceX.db")
cur = con.cursor()

In [3]:
%load_ext sql

In [4]:
%sql sqlite:///SpaceX.db

In [5]:
df = pd.read_csv("Spacex.csv")
df.to_sql("SPACEX_DATA", con, if_exists='replace', index=False,method="multi")

101

#### Display the names of the unique launch sites in the space mission

In [19]:
%sql SELECT DISTINCT Launch_Site FROM SPACEX_DATA

 * sqlite:///SpaceX.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


#### Display 5 records where launch sites begin with the string 'CCA'


In [12]:
%sql SELECT * FROM SPACEX_DATA WHERE Launch_Site LIKE 'CCA%'LIMIT(5)

 * sqlite:///SpaceX.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


#### Display the total payload mass carried by boosters launched by NASA (CRS)


In [16]:
%sql SELECT SUM(PAYLOAD_MASS__KG_) as Total_Payload_Mass FROM SPACEX_DATA WHERE Customer == 'NASA (CRS)'

 * sqlite:///SpaceX.db
Done.


Total_Payload_Mass
45596


#### Display average payload mass carried by booster version F9 v1.1


In [21]:
%sql SELECT AVG(PAYLOAD_MASS__KG_) as Avg_Payload_Mass FROM SPACEX_DATA WHERE Booster_Version =='F9 v1.1'

 * sqlite:///SpaceX.db
Done.


Avg_Payload_Mass
2928.4


In [48]:
%sql SELECT DISTINCT Launch_Site FROM SPACEX_DATA

 * sqlite:///SpaceX.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


#### List the date when the first successful landing outcome in ground pad was acheived.


In [31]:
%sql SELECT MIN(Date) AS FirstSuccessfull_landing_date FROM SPACEX_DATA WHERE [Landing _Outcome] == 'Success (ground pad)'

 * sqlite:///SpaceX.db
Done.


FirstSuccessfull_landing_date
01-05-2017


#### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [33]:
%sql SELECT Booster_Version FROM SPACEX_DATA WHERE [Landing _Outcome] = 'Success (drone ship)' AND PAYLOAD_MASS__KG_ > 4000  AND PAYLOAD_MASS__KG_ < 6000

 * sqlite:///SpaceX.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


#### List the total number of successful and failure mission outcomes


In [35]:
%sql SELECT (SELECT COUNT(*) FROM SPACEX_DATA WHERE [Landing _Outcome] LIKE 'Success%') as Success_outcomes, (SELECT COUNT(*) FROM SPACEX_DATA WHERE [Landing _Outcome] LIKE 'Failure%') as Failure_outcomes 

 * sqlite:///SpaceX.db
Done.


Success_outcomes,Failure_outcomes
61,10


#### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [37]:
%sql SELECT Booster_Version, PAYLOAD_MASS__KG_ FROM SPACEX_DATA WHERE PAYLOAD_MASS__KG_ == (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEX_DATA)

 * sqlite:///SpaceX.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


#### List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015


In [46]:
%sql SELECT Booster_Version, Launch_Site, [Landing _Outcome] FROM SPACEX_DATA WHERE [Landing _Outcome] LIKE 'Failure (drone ship)' AND Date >= '2015-01-01' AND Date <= '2015-12-31'

 * sqlite:///SpaceX.db
Done.


Booster_Version,Launch_Site,Landing _Outcome


#### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order


In [47]:
%sql SELECT [Landing _Outcome], COUNT([Landing _Outcome]) FROM SPACEX_DATA WHERE DATE >='2010-06-04' AND DATE <= '2017-03-20' GROUP BY [Landing _Outcome] ORDER BY COUNT([Landing _Outcome]) DESC

 * sqlite:///SpaceX.db
Done.


Landing _Outcome,COUNT([Landing _Outcome])


#### © IBM Corporation 2021. All rights reserved. <h3/>
